<div style ="border: blue solid 5px; font-family: Agency FB;	"><center>
<a href="#" ></a> 
    <h1> <font color="blue">Fusion des bases de données et segmentation des entreprises</font> </h1> 
<a href="#" ></a>     
</center>
</div>

Dans ce notebook nous effectuons la fusion des bases de données **Ademe**, **Siren**, **Diane**, **Esane** et **Citepa(Secten)**. Ensuite, nous mettons en place la segmentation des entreprises par catégorie selon la définition de l'INSEE.

## Table des matières

* [Packages](#1-chargement-des-packages)
* [Ademe + Siren](#2-fusion-siren-et-ademe)
* [Création de la durée d'activité](#3-création-de-la-durée-dactivité)
* [Fusion ADEME_SIREN avec DIANE](#4-fusion-ademesiren-avec-diane)
* [Fusion avec ESANE et SECTEN](#5-fusion-avec-esane-et-secten)
* [Segmentation](#6-segmentation)
* [Sauvegarde de la base](#sauvegarde-de-la-base)



## 1. Chargement des packages

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

# Warning
import warnings
warnings.filterwarnings("ignore")


In [2]:
os.chdir("D:/01_ENSAI/Sem_2/Projet_Statistique/projet-risque-de-transition")

## 2. Fusion SIREN et ADEME

In [3]:
## Importation des bases de données
siren = pd.read_csv("bdd/BaseSiren/siren_df.csv")
Ademe = pd.read_csv("bdd/Ademe/Ademe.csv")

In [4]:
Ademe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1860 entries, 0 to 1859
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 1860 non-null   float64
 1   organization_name  1860 non-null   object 
 2   staff              1860 non-null   float64
 3   reporting_year     1860 non-null   float64
 4   total_scope_1      1839 non-null   float64
 5   total_scope_2      1839 non-null   float64
 6   total_scope_3      1839 non-null   float64
 7   action_plan        1860 non-null   object 
 8   assessment_id      1860 non-null   float64
 9   legal_unit_id      1860 non-null   int64  
 10  evolution_scope_1  482 non-null    float64
dtypes: float64(8), int64(1), object(2)
memory usage: 160.0+ KB


In [5]:
siren.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1844 entries, 0 to 1843
Data columns (total 17 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   siren                                1844 non-null   int64  
 1   siret                                1844 non-null   int64  
 2   activitePrincipaleUniteLegale        1844 non-null   object 
 3   economieSocialeSolidaireUniteLegale  1692 non-null   object 
 4   caractereEmployeurUniteLegale        1844 non-null   object 
 5   trancheEffectifsUniteLegale          1825 non-null   object 
 6   anneeEffectifsEtablissement          1381 non-null   float64
 7   categorieEntreprise                  1803 non-null   object 
 8   codePostalEtablissement              1841 non-null   float64
 9   codeCommuneEtablissement             1841 non-null   object 
 10  dateCreationUniteLegale              1844 non-null   object 
 11  categorieJuridiqueUniteLegale 

**ADEME + SIREN**

In [6]:
Adem_Siren = Ademe.merge(siren, how = "left",left_on='legal_unit_id', right_on='siren', validate='1:1')

In [7]:
Adem_Siren.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1860 entries, 0 to 1859
Data columns (total 28 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   id                                   1860 non-null   float64
 1   organization_name                    1860 non-null   object 
 2   staff                                1860 non-null   float64
 3   reporting_year                       1860 non-null   float64
 4   total_scope_1                        1839 non-null   float64
 5   total_scope_2                        1839 non-null   float64
 6   total_scope_3                        1839 non-null   float64
 7   action_plan                          1860 non-null   object 
 8   assessment_id                        1860 non-null   float64
 9   legal_unit_id                        1860 non-null   int64  
 10  evolution_scope_1                    482 non-null    float64
 11  siren                         

In [8]:
Adem_Siren.drop('siren', axis = 1, inplace= True)

**Qualité de la fusion**

In [9]:
## Taux de perdition de la fusion SIREN - ADEME
taux_perte_Ademe = np.sum(Adem_Siren['siret'].isna())/Ademe.shape[0]*100
print("On obtient un taux de perdition de ", round(taux_perte_Ademe, 3), "%")
## Suppression de la non correspondance
Adem_Siren.drop(Adem_Siren.loc[Adem_Siren.siret.isna(),].index, axis=0, inplace=True)

On obtient un taux de perdition de  0.86 %


In [10]:
Adem_Siren.shape

(1844, 27)

## 3. Création de la durée d'activité

In [11]:
Adem_Siren[['annee_creation', 'mois_creation', 'jour_crea']] = Adem_Siren['dateCreationUniteLegale'].str.split('-', expand=True)
Adem_Siren[['annee_creation', 'mois_creation', 'jour_crea']] = Adem_Siren[['annee_creation', 'mois_creation', 'jour_crea']].astype(float)
Adem_Siren[['annee_creation', 'mois_creation', 'jour_crea']].dtypes
Adem_Siren['Duree_Activite'] = Adem_Siren['reporting_year'] - Adem_Siren['annee_creation']
Adem_Siren['Duree_Activite'].describe()

count    1844.000000
mean       38.480477
std        28.711035
min        -1.000000
25%        19.000000
50%        31.000000
75%        52.000000
max       120.000000
Name: Duree_Activite, dtype: float64

**Variable de fusion avec Diane**

In [12]:
Adem_Siren['legal_unit_id']= Adem_Siren['legal_unit_id'].apply(str)
Adem_Siren['reporting_year'] = Adem_Siren['reporting_year'].apply(int)
Adem_Siren['SIREN_YEAR'] = Adem_Siren['legal_unit_id'].apply(str) + Adem_Siren['reporting_year'].apply(str)


In [13]:
Adem_Siren['SIREN_YEAR'].dtype

dtype('O')

## 4. Fusion ADEME_SIREN avec DIANE

**Importation DIANE**

In [14]:
diane = pd.read_csv('bdd/diane/Finale/diane.csv')
diane.shape

(1605, 640)

In [15]:
diane['SIREN_YEAR'] = diane['SIREN_YEAR'].apply(int).apply(str)

In [16]:
ASD = Adem_Siren.merge(diane, how='left',on = 'SIREN_YEAR', validate='1:1')

**Qualité de la jointure**

In [17]:
## Taux de perdition de la fusion SIREN - ADEME
taux_perteASD = ((ASD['siren'].isna()) & (ASD.categorieEntreprise!='PME')).mean()
print("On obtient un taux de perdition de ", round(taux_perteASD, 4)*100, "%", 
'soit ', ASD['siren'].isna().sum(), 'Entreprises ont aucune correspondance')


On obtient un taux de perdition de  33.35 % soit  674 Entreprises ont aucune correspondance


In [18]:
# On retient les entreprises ayant une correspondance dans les deux bases ou qui soit PME même s'il n'ont pas de correspondance
ASD.drop(ASD.loc[(ASD['siren'].isna()) & (ASD.categorieEntreprise!='PME'),].index, axis=0, inplace= True)

In [19]:
ASD.shape

(1229, 671)

## 5. Fusion avec ESANE et SECTEN

In [20]:
base_filiere = pd.read_csv('bdd/Esane/base_secteur_vf.csv')

In [21]:
base_filiere.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 26 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   annee_secteur                            216 non-null    int64  
 1   NbEntrp_secteur                          216 non-null    float64
 2   Activite_secteur                         216 non-null    object 
 3   SecteurDActivite_secteur                 216 non-null    object 
 4   FraisPersonnel_secteur                   216 non-null    float64
 5   Salaries_tps_plein_secteur               216 non-null    float64
 6   CA_exportation_secteur                   216 non-null    float64
 7   CA_hors_taxe_secteur                     216 non-null    float64
 8   EBE_secteur                              216 non-null    float64
 9   VA_et_autres_prdts_charges_secteur       216 non-null    float64
 10  ResultatNetComptable_secteur             216 non-n

In [22]:
base_filiere.SecteurDActivite_secteur.value_counts().sort_index()

A    12
B    12
C    12
D    12
E    12
F    12
G    12
H    12
I    12
J    12
K    12
L    12
M    12
N    12
P    12
Q    12
R    12
S    12
Name: SecteurDActivite_secteur, dtype: int64

In [23]:
ASD.APE_niv_1.value_counts().sort_index()

A      4
B      2
C    423
D      9
E     28
F     95
G    184
H     98
I     30
J     92
K     17
L     17
M    106
N     83
O      1
Q     24
R      2
S     14
Name: APE_niv_1, dtype: int64

In [24]:
base_filiere.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 26 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   annee_secteur                            216 non-null    int64  
 1   NbEntrp_secteur                          216 non-null    float64
 2   Activite_secteur                         216 non-null    object 
 3   SecteurDActivite_secteur                 216 non-null    object 
 4   FraisPersonnel_secteur                   216 non-null    float64
 5   Salaries_tps_plein_secteur               216 non-null    float64
 6   CA_exportation_secteur                   216 non-null    float64
 7   CA_hors_taxe_secteur                     216 non-null    float64
 8   EBE_secteur                              216 non-null    float64
 9   VA_et_autres_prdts_charges_secteur       216 non-null    float64
 10  ResultatNetComptable_secteur             216 non-n

In [25]:
ASD = ASD.merge(base_filiere, left_on =['reporting_year','APE_niv_1'],
right_on = ['annee_secteur','SecteurDActivite_secteur'], how='left', validate='m:1')

In [26]:
ASD['secteur_secteur'] = ASD['secteur_secteur'].fillna('Autre')
ASD.loc[ASD.secteur_secteur == "agriculture", 'secteur_secteur'] = "Autre"

**Qualité de la fusion**

Le secteur O est absent dans ESANE et le secteur P n'est pas présent dans ADEME. Après fusion on a une perte de 99 entreprises. Cette perte n'affectera que la modélisation Top-Down.

## 6. Segmentation

**Imputation du nombre d'employés**

Enfin de pouvoir faire la segmentation avec le nombre d'employés fourni par **Diane**, nous imputons ces valeurs manquantes à l'aide de la variable **staff** fournie par **Ademe**

In [27]:
## Nombre d'employe imputé avec staff de Ademe
pd.set_option('display.max_rows', None)
ASD[['Average number of employees', 'staff']]

ASD.loc[ASD['Average number of employees'].isna(),
 'Average number of employees'] = ASD.loc[ASD['Average number of employees'].isna(),
 'staff']

**Segmentation selon la définition de l'INSEE**

In [28]:
## TPE/PME
cond1 = (ASD['Average number of employees']<=250) & ((ASD['NetTurnover']<=3000000) |
 (ASD['TotalAsset']<= 43000000)) #1 - 5
ASD["classe_temp1"] = np.select([cond1, ~cond1], [1, 5])

## ETI
cond2 = (ASD['Average number of employees']<=1500) & ((ASD['NetTurnover']<=1500000000) |
 (ASD['TotalAsset']<= 2000000000)) #2 - 5
ASD['classe_temp2'] = np.select([cond2, ~cond2], [2, 5])

## GE
cond3 = (ASD['Average number of employees']>1500) | ((ASD['NetTurnover']>1500000000) & 
(ASD['TotalAsset']> 2000000000)) #3 - 5
ASD['classe_temp3'] = np.select([cond3, ~cond3], [3, 5])

## Classement final selon l'INSEE
ASD['classe'] = ASD[['classe_temp1', 'classe_temp2', 'classe_temp3']].min(axis=1)
ASD.drop(['classe_temp1', 'classe_temp2', 'classe_temp3'], axis = 1, inplace = True)
## Labélisation
ASD['classe'] = ASD['classe'].apply(str).replace(['1', '2', '3', '5'], ['TPE/PME', 'ETI', 'GE', 'None'])

In [29]:
## Les non classés sont remplacé par le classement de SIREN
ASD.loc[ASD.classe == 'None', 'classe' ] = ASD.loc[ASD.classe == 'None', 'categorieEntreprise' ]
ASD['classe'] = ASD['classe'].replace('PME', 'TPE/PME')


In [30]:
ASD.classe.value_counts()

ETI        817
GE         315
TPE/PME     97
Name: classe, dtype: int64

**A partir de quel effectif peut on dire qu'un ETI est semblable à une TPE/PME**

In [31]:
## Comparaison des moyennes pour la ventilation des ETI.
from scipy import stats
## Ceci ne concerne que les entreprises ayant une correspondance avec DIANE
ASD['NetTurnover2'] = ASD['NetTurnover'].fillna(ASD['NetTurnover'].mean())
df_test = ASD[(~ASD.siren.isna())].copy()
taille = []
for seuil in range(400, 1000) :
   cond1_eti = (df_test.classe == 'ETI') & (df_test['Average number of employees']<= seuil)

   
   result = stats.ttest_ind(df_test.loc[cond1_eti, 'NetTurnover2'], 
                           df_test.loc[df_test.classe=='TPE/PME', 'NetTurnover2'])
   if result[1]>=0.01:
      taille.append(seuil)

ASD.drop('NetTurnover2', axis=1, inplace=True)   
## Taille maximale telle que ETI == TPE/PME
np.max(taille)

505

Jusqu'au seuil 505 les TPE/PME et les ETI ne sont pas significativement différentes au sens du chiffre d'affaire. Donc nous allons ventiler les ETI d'effectif inférieur à 505 vers les TPE/PME

In [32]:

cond1_eti = (ASD.classe == 'ETI') & (ASD['Average number of employees']<=505)
sum(cond1_eti)
ASD.loc[cond1_eti, 'classe'] = 'TPE/PME'

cond2_eti = (ASD.classe == 'ETI') & (ASD['Average number of employees']>505)
sum(cond2_eti)

ASD.loc[cond2_eti, 'classe'] = 'GE'

In [33]:
ASD.classe.value_counts()

GE         1026
TPE/PME     203
Name: classe, dtype: int64

In [34]:
pd.crosstab(ASD.categorieEntreprise, ASD.classe, margins=True)

classe,GE,TPE/PME,All
categorieEntreprise,,,
ETI,429,75,504
GE,586,50,636
PME,6,77,83
All,1021,202,1223


**Renommination des variables**

In [35]:
symbole = [
   '#',
   '@',
   '&',
   '(',
   ')',
   '!',
   '/',
   '%',
   '.\xa0',
   ':',
   '.',
   '-',
   '+',
   '*',
   ':',
   ','
   ]
for s in symbole:
   ASD.columns=ASD.columns.str.replace(s,'')
ASD.columns=ASD.columns.str.replace(" ","_")



## Sauvegarde de la base

In [36]:
ASD.to_csv("bdd/Base_finale/ASDES.csv", index=False)